# Subset surface climate data for eastern Australia

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [2]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = '00:30:00'
cores = 48
memory = str(4 * cores) + 'GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=['-q normal',
                                           '-P w42',
                                           '-l ncpus='+str(cores),
                                           '-l mem='+str(memory),
                                           '-l storage=gdata/w42+gdata/rt52'],
                     local_directory='$TMPDIR',
                     job_directives_skip=["select"])
                     # python=os.environ["DASK_PYTHON"])

In [3]:
cluster.scale(jobs=5)
client = Client(cluster)

In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.41.69:8787/status,
Dashboard: http://10.6.41.69:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.41.69:44325,Workers: 0
Dashboard: http://10.6.41.69:8787/status,Total threads: 0
Started: 2 minutes ago,Total memory: 0 B


In [5]:
# client.close()
# cluster.close()

In [6]:
# %load_ext autoreload
# %autoreload 2

In [5]:
import xarray as xr
import numpy as np

In [6]:
import functions as fn

In [7]:
rez_region = {
    'name': 'REZ',
    'boundary': fn.get_REZ_boundary()
}
aus_region = {
    'name': 'AUS',
    'boundary': [112, 155, -10, -45]
}
extended_aus_region = {
    'name': 'extended_AUS',
    'boundary': fn.get_extended_AUS_boundary()
}

# ERA5

In [8]:
years = range(1959, 2022)

In [9]:
root_path = '/g/data/rt52/era5/single-levels/'

# Solar radiation

Ideally we would have `ssrd` [Joules m^-2]. However, NCI does not store this variable.

Instead, (I think) we can use the Mean surface downward short-wave radiation flux `msdwswrf` [Watts m^-2]. This is the same as `ssrd` but expressed as a temporal average. In this case the average radiation per second over the hour.

Links:

- https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Table4
- https://apps.ecmwf.int/codes/grib/param-db?id=169
- https://apps.ecmwf.int/codes/grib/param-db?id=235035
- https://www.ecmwf.int/sites/default/files/elibrary/2015/18490-radiation-quantities-ecmwf-model-and-mars.pdf

In [9]:
mssrd = fn.open_era_data(
    root_path=root_path+'reanalysis/',
    variable='msdwswrf',
    years=years,
    subset_region=aus_region['boundary'],
    rename_lon_lat=['lon', 'lat']
)

Check size of dataset (in GB)

In [20]:
mssrd.nbytes / (1024 ** 3)

50.18762895464897

Re-chunk to ~100Mb chunks.

In [11]:
mssrd = mssrd.chunk({
    'time': 24*90,
    'lon': -1,
    'lat': -1
})

Save to zarr

In [12]:
mssrd_encoding = {
    'msdwswrf': {
        'dtype': 'float32'
    }
}

In [19]:
mssrd.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/msdwswrf/msdwswrf_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=mssrd_encoding
)

### Compute daily averages

Do this using 24 hour rolling mean, then select all 1400 hour times (equivalent to 0000 eastern state time)

In [21]:
mssrd = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/msdwswrf/msdwswrf_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+aus_region['name']+'_region.zarr',
    consolidated=True
)

In [ ]:
mssrd_1400 = fn.daily_mean_1400(mssrd)

In [22]:
# mssrd_24hr = mssrd.rolling(time=24).mean()

In [23]:
# mssrd_1400 = mssrd_24hr.isel(time=mssrd_24hr.time.dt.hour == 14)

In [24]:
mssrd_1400 = mssrd_1400.chunk({'time': 365*6})

In [25]:
mssrd_1400.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/msdwswrf/msdwswrf_era5_daily_1400UTC_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=mssrd_encoding
)

Calculate using $w = \sqrt{u^2 + v^2}$

We do this for the extended Aus region as we use these variables to assess wider-scale synoptics

In [13]:
u = fn.open_era_data(
    root_path=root_path+'reanalysis/',
    variable='100u',
    years=years,
    subset_region=extended_aus_region['boundary'],
    rename_lon_lat=['lon', 'lat']
)

In [14]:
v = fn.open_era_data(
    root_path=root_path+'reanalysis/',
    variable='100v',
    years=years,
    subset_region=extended_aus_region['boundary'],
    rename_lon_lat=['lon', 'lat']
)

In [32]:
u = u.chunk({
    'time': 24*21,
    'lon': -1,
    'lat': -1
})

v = v.chunk({
    'time': 24*21,
    'lon': -1,
    'lat': -1
})

In [10]:
u_encoding = {
    'u100': {
        'dtype': 'float32'
    }
}
v_encoding = {
    'v100': {
        'dtype': 'float32'
    }
}

In [ ]:
u.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/100u/100u_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=u_encoding
)

In [44]:
v.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/100v/100v_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=v_encoding
)

Process wind speed

In [62]:
u = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/100u/100u_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    consolidated=True
)
v = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/100v/100v_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    consolidated=True
)

In [63]:
w = np.sqrt(u.rename({'u100': 'w100'})**2 + v.rename({'v100': 'w100'})**2)

Check size of dataset (in GB)

In [64]:
w.nbytes / (1024 ** 3)

185.0004643537104

Save to zarr

In [11]:
w_encoding = {
    'w100': {
        'dtype': 'float32'
    }
}

In [68]:
w.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/100w/100w_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=w_encoding
)

In [69]:
client.restart()

<Client: 'tcp://10.6.42.64:45541' processes=0 threads=0, memory=0 B>

### Compute daily averages

### Note: there must be a better way! Sometimes this kills workers when I have 5/6 full nodes. Am I chunking wrong? Should I process for each year separately?

### Remember to wait for all requested nodes to come online... seems ok with 4 nodes. Maybe possible with fewer...

Do this using 24 hour rolling mean, then select all 1400 hour times (equivalent to 0000 eastern state time)

In [12]:
u = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/100u/100u_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    consolidated=True
)
v = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/100v/100v_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    consolidated=True
)

In [17]:
w = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/100w/100w_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    consolidated=True
)

In [95]:
u_1400 = fn.daily_mean_1400(u)
u_1400 = u_1400.chunk({'time': 365*6})

In [13]:
v_1400 = fn.daily_mean_1400(v)
v_1400 = v_1400.chunk({'time': 365*6})

In [18]:
w_1400 = fn.daily_mean_1400(w)
w_1400 = w_1400.chunk({'time': 365*6})

In [99]:
u_1400.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/100u/100u_era5_daily_1400UTC_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=u_encoding
)

In [14]:
v_1400.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/100v/100v_era5_daily_1400UTC_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=v_encoding
)

In [19]:
w_1400.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/100w/100w_era5_daily_1400UTC_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=w_encoding
)

# Temperature 2m

In [12]:
t = fn.open_era_data(
    root_path=root_path+'reanalysis/',
    variable='2t',
    years=years,
    subset_region=extended_aus_region['boundary'],
    rename_lon_lat=['lon', 'lat']
)

Check size of dataset (in GB)

In [13]:
t.nbytes / (1024 ** 3)

185.0004643537104

Re-chunk to ~100Mb chunks.

In [14]:
t = t.chunk({
    'time': 24*21,
    'lon': -1,
    'lat': -1
})

Save to zarr

In [15]:
t_encoding = {
    't2m': {
        'dtype': 'float32'
    }
}

In [16]:
t.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/2t/2t_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=t_encoding
)

### Compute daily averages

Do this using 24 hour rolling mean, then select all 1400 hour times (equivalent to 0000 eastern state time)

In [22]:
t = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/2t/2t_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    consolidated=True
)

In [23]:
t_1400 = fn.daily_mean_1400(t)
t_1400 = t_1400.chunk({'time': 365*6})

In [24]:
t_1400.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/2t/2t_era5_daily_1400UTC_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=t_encoding
)

### Compute anomalies

In [10]:
t_1400 = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/2t/2t_era5_daily_1400UTC_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    consolidated=True
)

In [11]:
t_1400_anoms = t_1400.groupby('time.dayofyear').apply(lambda x: x - x.mean('time'))

In [13]:
%%time
t_1400_anoms.to_zarr( # Write
    '/g/data/w42/dr6273/work/data/era5/2t/2t_anoms_era5_daily_1400UTC_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+extended_aus_region['name']+'_region.zarr',
    mode='w',
    consolidated=True
)

CPU times: user 1min 17s, sys: 3.72 s, total: 1min 21s
Wall time: 5min 1s


# Monthly runoff

In [30]:
ro = fn.open_era_data(
    root_path=root_path+'monthly-averaged/',
    variable='ro',
    years=years,
    subset_region=rez_region['boundary'],
    rename_lon_lat=['lon', 'lat']
)

Check size of dataset (in GB)

In [31]:
ro.nbytes / (1024 ** 3)

0.03534848242998123

Re-chunk to single chunk.

In [32]:
ro = ro.chunk({
    'time': -1,
    'lon': -1,
    'lat': -1
})

Save to zarr

In [33]:
ro_encoding = {
    'ro': {
        'dtype': 'float32'
    }
}

In [35]:
ro.to_zarr(
    '/g/data/w42/dr6273/work/data/era5/ro/ro_era5_monthly-averaged_sfc_'+str(years[0])+'-'+str(years[-1])+'_'+rez_region['name']+'_region.zarr',
    mode='w',
    consolidated=True,
    encoding=ro_encoding
)